In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names, but [a-zA-Z]+ was fitted with feature names",
    category=UserWarning,
)

warnings.filterwarnings = lambda *a, **kw: None
from IPython.core.display import HTML

HTML(open("custom.html", "r").read())

# Chapter 5: Preprocessing, pipelines and hyperparameters optimization

## Types of data transformations / preprocessing

Note: we use terms data *transformation* and *preprocessing* interchangeably.

We've seen before that adding polynomial features to the 2D `xor` and `circle` problem made both tasks treatable by a simple linear classifier.

Beyond adding polynomial features, there are other important preprocessors / transformers to know of.

### Scaler

A scaler applies a linear transformation on every feature. Those transformations are individual per column.

The two most important scalers in `sklearn.preprocessing` module are:

- `MinMaxScaler`: after applying this scaler, the minumum in every column is 0, the maximum is 1.

- `StandardScaler`: scales columns to mean value 0 and standard deviation 1.

#### Why scaling?

Let us assume we have two features `x` and `y` with different ranges: `x` is in the range `0` to `3` and `y` in `0` to `1`: We have sampled both variables with the same resolution:

<img src="images/different_scales.png">

Some classifiers like `SVC` and `KNeighborsClassifier` use Euclidian distances between features internally. They assume that features which are close in distance also have similar classes or target values.

So let us check what features have an Euclidian distance to `(1.5, 0.5)` below `0.2`:

<img src="images/before_rescaling.png"/>

As you can see the points in the circle cover a big range of `y` values, but only a few points from `x`. You can assume that the feature `y` has a strong influence on how a classifier works, and `x` has less influence.

You can also see that on absolute scales the circle is not a circle:

<img src="images/before_rescaling_skewed_circle.png"/>


When we scale `x` to the range `0` to `1` the situation is as follows:

<img src="images/after_rescaling.png" >


The reason to use a scaler is to compensate

- for different orders of magnitudes
- and physical units

of the features.

We say that a machine learning methods or transformations is **scale invariant** when rescaling (i.e. multiplying) any of the features (columns) by a nonzero number does not change predictions or transformation results, but may change the method or transformation itself.

For instance, a simple linear classifier `weight_x * x + weight_y * y = threshold` is scale invariant because when changing unit a feature e.g. from centimeters to meters you divide feature values by `100`, but multiplying a feature weight by `100` gives the exact same model: `(weight_x * 100) * (x / 100) + weight_y * y = threshold`.

<div class="alert alert-block alert-warning">
<p><i class="fa fa-warning"></i>
    <strong>Always (re)scale features</strong> before using non-scale invariant:
</p>
<ul>
    <li>distance-based methods, like <code>SVC</code> or <code>KNeighborsClassifier</code>,</li>
    <li>non-linear transformations or methods using such transformations, like <code>PolynomialFeatures</code> or <code>LogisticRegression</code> classifier,</li>
    <li>otherwise, non-linearly scale-dependent transformations or methods, like <code>PCA</code>, or <code>l1</code> and <code>l2</code> regularization penalty-based methods, like <code>Ridge</code> or <code>Lasso</code> regression.</li>
</ul>
<p><i class="fa fa-warning"></i>
    <strong>Never (re)scale</strong> your features if they are already in the same scale (e.g. pixel values in an image), as any almost-constant unimportant features may gain importance.
</p>
</div>

### Dimensionality reduction (PCA)

Reducing the dimensionality of a multi variate data set removes redundancies in it, such as highly correlated columns. We've discussed before that reducing redundancy and noise can help to avoid overfitting.

One of the most effective techniques for dimensionality reduction is a Principal Component Analysis (PCA). Its biggest downside is that the resulting few new features (principal components) cannot be directly interpreted in terms of original features.

<table>
    <tr><td><img src="images/pca-3d_to_2d.jpg" width="725px"/></td></tr>
    <tr><td><center><sub>Source: <a href="https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e">https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e</a></sub></center></td></tr>
</table>

The `sklearn.decomposition` module contains the standard `PCA` utility, as well as many of its variants and other dimensionality reduction techniques, and some more general features matrix decomposition techniques.

**Don't forget to scale features before using PCA.**

<div class="alert alert-block alert-info">
<p><i class="fa fa-info-circle"></i>
Dimensional reduction using PCA consists of finding the features that maximize the variance. If one feature varies more than the others only because of their respective scales, PCA would determine that such feature dominates the direction of the principal components. Thus, in order to work correctly PCA requires features to be in the same scale, i.e. with comparable (unit) variances. To that end, one typically uses the <code>StandardScaler</code> transformation. Alternatively, use <code>RobustScaler</code> when dealing with outliers or <code>MinMaxScaler</code> when it is required to capture a small variance in features. More information and examples can be found here: <a href="https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#effect-of-rescaling-on-a-pca-dimensional-reduction">https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#effect-of-rescaling-on-a-pca-dimensional-reduction</a>
<p>
</div>

### Function transformers

It can help to apply functions like `log` or `exp` or `1/x` to features to improve classification performance.

Lets assume you want to forecast the outcome of car crash experiments and one variable is the time $t$ needed for the distance $l$ from start to crash. Transforming this to the actual speed $\frac{l}{t}$ could be a more informative feature then $t$.

Use a `FunctionTransformer` utility from `sklearn.preprocessing` to define and apply a function transformer.

### Missing values imputers

Sometimes data contain missing values. Data imputation is a strategy to fill up missing values. Standard in statistics Missing (Completely/Not) At Random (MAR; MCAR; MNAR) approaches are not well-suited for machine learning tasks. Instead, in `sklearn.impute` module you will find:

* `SimpleImputer`: columnwise mean/median/most frequent value approach that works great with good classifier and a lot of non-missing data, otherwise use
* (semi-supervised) machine learning imputers:
    * `KNNImputer`: mean value from k-Nearest Neighbors (closest samples by non-missing feature values); note: do scale features before using it,
    * `IterativeImputer`: regresses each feature with missing values on other features, in an iterated round-robin fashion over each feature.

## A scaling example

As an example we demonstrante how a scaler can be implemented. Our scaling strategy will scale given values to the range 0 to 1.

First we create a random data matrix and compute columnwise min and max values:

In [ ]:
import numpy as np

# for reproducible numbers:
np.random.seed(42)

values = np.random.random((5,)) * 20 - 10

min_value = values.min()
max_value = values.max()

print("values:", values)
print()
print("min value:", min_value)
print("max value:", max_value)

The strategy for scaling is as follows: Our values $v$ are in the range $v_{min}$ to $v_{max}$:

$$
v_{min} \le  v  \le v_{max}
$$


Then subtracting $v_{min}$ results in 

$$
0 \le  v - v_{min} \le v_{max} - v_{min}
$$
  
Finally dividing by the right hand side delivers the property we are looking for:

$$
0 \le \frac{v - v_{min}}{v_{max} - v_{min}} \le 1
$$


In Python:

In [ ]:
scaled_values = (values - min_value) / (max_value - min_value)

print("scaled values:", scaled_values)

You can see that all values are now scaled as intended.

To apply the same strategy column per column to a feature matrix, `scikit-learn` offers a `MinMaxScaler`:

In [ ]:
features = np.random.random((5, 3)) * 20 - 10
print(features)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 1. learning -> determine columnwise min/max values
scaler = MinMaxScaler().fit(features)

# 2. transformation -> apply linear transformation based on min/max values:
print(scaler.transform(features))

In [ ]:
# 1+2. shorter
print(scaler.fit_transform(features))

## How to correctly transform data?

We can divide preprocessing into two classes:

1. Preprocessing which depends on the **full feature matrix** (including column-wise preprocessing). E.g.

   - Dimensionality reduction (PCA)
   - Column-wise functional transforms or scaling
   - Imputation of missing values

2. Preprocessing which can be applied **row per row individually**. E.g.

   - Adding polynomial features
   - Row-wise functional transforms or scaling (e.g. when a row represents an image and we want to compensate for different illumination)

<div class="alert alert-block alert-warning">
    <p style="font-size: 150%;"><i class="fa fa-info-circle"></i>&nbsp;Data preprocessing commandments<p>
    <ol>
        <li>When we include preprocessing in a classification approach, <strong>we must apply exactly the same preprocessing on new incoming data</strong>!</li>
        <li>This implies that <strong>for preprocessors which depend on the full data set we must never preprocess data before cross-validation</strong>!</li>
    </ol>
</div>

Running preprocessors on the full dataset before cross-validation lets information of "unseen" test data sneak into the classifier.

<img src="./images/2xi5wt.jpg" width=50%/>

<p style="font-size: 150%;"> This is how we must proceed</p>

In case for the `MinMaxScaler` transformer:

1. Determine column-wise minimum und maximum values of the training features.
2. Use these min/max values to scale training data.
3. Learn classifier `C` on the scaled training data.


4. Use values from 1. to scale evaluation data (thus, we might create values outside `0..1`).
5. Apply classifier `C` to the scaled evaluation data.
6. Assess `C` performance.

In general:

1. Learn transformer `P` on the training data.
2. Apply `P` to the training data.
3. Learn classifier `C` on the transformed training data.


4. Apply `P` from 1. to the evaluation data.
5. Apply classifier `C` to the transformed evaluation data.
6. Assess `C` performance.

<div class="alert alert-success" role="alert">

## Take-home message: Preprocessing    

| Transformation | When is it applied? |
| ------ | ------ |
| Scaling | - bring features to same value range <br> - especially important when using models which use Euclidean distance between features | 
| PCA | - dimensionality reduction of multi-dimensional dataset  |
| Function transformer | - transform present feature into more informative feature  |
| Missing values imputers | - when single datapoints are missing |

- **Dependency of preprocessing methods on the dataset**: Preprocessing methods either depend on the full feature matrix or only on single rows
- **Order of data transformations**: Always split your data into train and evaluation set and then apply preprocessing and model training.

    
<p>
</div>

## The scikit-learn API (quick recap)

We've seen before that we can swap `scikit-learn` classifiers easily without changing much code. 

This is possible, because all classifiers have methods `.fit` and `.predict` which also have the same function signature (this means number and meaning of arguments is always the same for every implementation of `.fit` respectively `.predict`.)

This consistent design within `scikit-learn` also applies to data transformers, which all have methods`.fit`, `.transform` and `.fit_transform`.

This consistent API allows setting up easily processing pipelines.

## Pipelines

A so called **classification pipeline consists of 0 or more data transformers plus a final classifier**.

Let us start with the following pipeline:

1. Apply scaling to mean 0 and std deviation 1
2. Use PCA to reduce data to 3 dimensions
3. Train `SVC` classifier.




In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

p = make_pipeline(StandardScaler(), PCA(3), SVC())

<div class="alert alert-block alert-warning">
<p><i class="fa fa-info-circle"></i>
A pipeline "behaves" like a single classifier - it implements <code>.fit()</code> and <code>.predict()</code> methods.</p>
</div>


In [ ]:
print("p.fit    ", p.fit is not None)
print("p.predict", p.predict is not None)

Because of this we can also use cross-validation in the same way as we did before:

In [ ]:
import pandas as pd

beer_data = pd.read_csv("data/beers.csv")

features = beer_data.iloc[:, :-1]
labels = beer_data.iloc[:, -1]


from sklearn.model_selection import cross_val_score

print(cross_val_score(p, features, labels, scoring="accuracy", cv=5).mean())

<div class="alert alert-block alert-warning">

<i class="fa fa-info-circle"></i>&nbsp;One benefit of using a pipeline is that you will not mistakenly scale the full data set first, instead we follow the strategy we've described above automatically.

</div>

Bonus: you can easily visualize a more complex pipelines:

In [ ]:
from sklearn import set_config

set_config(display="diagram")
p

### How to setup a good pipeline?

Regrettably there is no recipe how to setup a good performing classification pipeline except reasonable preprocessing, especially feature engineering. After that it is up to experimentation and the advice on how to choose classifiers we gave in the last script.

Let us try out different pipeplines and evaluate them:

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler
from sklearn.svm import SVC

for p in [
    make_pipeline(KNeighborsClassifier()),
    make_pipeline(MinMaxScaler(), KNeighborsClassifier()),
    make_pipeline(StandardScaler(), KNeighborsClassifier()),
    make_pipeline(SVC()),
    make_pipeline(StandardScaler(), SVC()),
    make_pipeline(LogisticRegression()),
    make_pipeline(StandardScaler(), LogisticRegression()),
    make_pipeline(StandardScaler(), PCA(2), LogisticRegression()),
    make_pipeline(StandardScaler(), PCA(3), LogisticRegression()),
    make_pipeline(PolynomialFeatures(2), SVC()),
    make_pipeline(StandardScaler(), PolynomialFeatures(2), SVC()),
    make_pipeline(StandardScaler(), PolynomialFeatures(3), SVC()),
    # tech: max_iter to prevent a convergence warning
    make_pipeline(PolynomialFeatures(2), LogisticRegression(max_iter=10000)),
    make_pipeline(StandardScaler(), PolynomialFeatures(2), LogisticRegression()),
]:

    print(
        "{:.3f}".format(
            cross_val_score(p, features, labels, scoring="accuracy", cv=5).mean()
        ),
        end=" ",
    )
    print([pi[1] for pi in p.steps])

### Exercise session

1. Can you come up with a better performing classification pipeline?

In [ ]:
# SOLUTION

for p in [
    # previouly best pipelines
    make_pipeline(StandardScaler(), PolynomialFeatures(), LogisticRegression()),
    make_pipeline(StandardScaler(), SVC()),
    # bit better
    make_pipeline(
        StandardScaler(), PolynomialFeatures(), PCA(10), LogisticRegression()
    ),
    make_pipeline(StandardScaler(), SVC(C=20, gamma=0.1)),
]:

    print(
        "{:.3f}".format(
            cross_val_score(p, features, labels, scoring="accuracy", cv=5).mean()
        ),
        end=" ",
    )
    print([pi[1] for pi in p.steps])

####  Optional exercises

1. Build a classification pipeline to classifiy the 2D xor- and circle data sets with linear classifiers. Also assess their performance.

2. Build a classification pipeline for the digits data set. This data set was described in the first script where we've shown how to flatten images to feature vectors. Experiment with `PCA` preprocessing, `ScandardScaler` and `SVC`. Show a few of the images with correct and incorrect labels.

In [ ]:
# SOLUTION
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC


def check_pipelines(data):
    features = data.iloc[:, :-1]
    labels = data.iloc[:, -1]

    for p in [
        make_pipeline(StandardScaler(), LogisticRegression()),
        make_pipeline(StandardScaler(), LinearSVC()),
        make_pipeline(StandardScaler(), PolynomialFeatures(2), LogisticRegression()),
        make_pipeline(StandardScaler(), PolynomialFeatures(2), LinearSVC()),
    ]:

        print(
            "{:.3f}".format(
                cross_val_score(p, features, labels, scoring="accuracy", cv=5).mean()
            ),
            end=" ",
        )
        print([pi[1] for pi in p.steps])


print("#### XOR")
xor_data = pd.read_csv("data/xor.csv")
check_pipelines(xor_data)
print()
print("#### Circle")
circle_data = pd.read_csv("data/circle.csv")
check_pipelines(circle_data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler
from sklearn.svm import SVC

# classifying digits

data_set = load_digits()
features = data_set.data
labels = data_set.target

for p in [
    make_pipeline(StandardScaler(), SVC()),
    make_pipeline(PCA(10), SVC()),
    make_pipeline(StandardScaler(), PCA(10), SVC()),  # no? pixels are already scaled
    make_pipeline(
        PCA(10), StandardScaler(), SVC()
    ),  # no? already scaled, even after PCA
    make_pipeline(PCA(15), SVC()),
    make_pipeline(PCA(17), SVC()),
    make_pipeline(PCA(20), SVC()),
    make_pipeline(PCA(17), SVC(C=0.5)),
    make_pipeline(PCA(17), SVC(C=2)),
    make_pipeline(PCA(17), SVC(C=4)),
    make_pipeline(PCA(17), SVC(C=5)),
    make_pipeline(PCA(17), StandardScaler(), SVC(C=4)),  # no? no, just a sanity check
]:

    print(
        "{:.3f}".format(
            cross_val_score(p, features, labels, scoring="accuracy", cv=5).mean()
        ),
        end=" ",
    )
    print([pi[1] for pi in p.steps])


from sklearn.model_selection import train_test_split

# split 80:20 with fixed randomization:
(features_train, features_test, labels_train, labels_test) = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

p.fit(features_train, labels_train)
predicted = p.predict(features_test)
incorrect = np.where(predicted != labels_test)[0]

print("#### Incorrect")
print(incorrect)


def show(indices):
    plt.figure(figsize=(10, 7))

    for i, idx in enumerate(indices):
        plt.subplot(1, len(indices), i + 1)
        img = features_test[idx].reshape(8, 8)
        plt.imshow(
            img,
            cmap="gray",
        )
        plt.title("is {}, predicted as {}".format(labels_test[idx], predicted[idx]))


show(incorrect)

# there are many correct ones, so let's pick a sample of size of the numer of incorrect ones
import random

correct = np.where(predicted == labels_test)[0]
correct = random.sample(
    list(correct), len(incorrect)
)  # from np array to list, this is required!
show(correct)

<div class="alert alert-block alert-info"><p>
<i class="fa fa-info-circle"></i>&nbsp;
Up to now we've applied preprocessing only to the full features table. <strong>To preprocess single columns or a subset of them, e.g. to apply function transformers, or to input missing values, or to encode categorical columns use a <code>ColumnTransformer</code> utility</strong>. A good overview is given in <a href="https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html">a tutorial on applying <code>ColumnTransformer</code>s to mixed-type columns</a>.
</p></div>


## Reusing pipelines: persist on a disk

Learning and finding a good performing pipeline can be time intensive. It makes sense to **store and reuse pipeline later for predictions**.

To that end, **use a standard library Python module `pickle` to serialize and deserialize a pipeline object** (or any other "serializable" Python object, including a classifier itself, or hyper-parameters search result object):

In [ ]:
import os
import pickle
import tempfile

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

beer_data = pd.read_csv("data/beers.csv")

features = beer_data.iloc[:, :-1]
labels = beer_data.iloc[:, -1]


p = make_pipeline(StandardScaler(), PCA(3), SVC())
p.fit(features, labels)
print("SVC learnt intercept:\n", p.named_steps["svc"].intercept_)


# ".pkl" is the standard file extension for "pickled" objects
p_path = os.path.join(tempfile.mkdtemp(), "pipeline.pkl")


# open file to _w_rite _b_inary data
with open(p_path, "wb") as p_file:
    pickle.dump(p, p_file)

In [ ]:
# open file to _r_ead _b_inary data
with open(p_path, "rb") as p_file:
    p_loaded = pickle.load(p_file)
print("SVC learnt intercept (loaded):\n", p.named_steps["svc"].intercept_)

### Caching pipeline data transformers

Another common case is **reusing preprocessing steps that are already fitted to data - the data transformers**. This is useful when trying out pipeline composition variations, or when optimizing hyper-parameters of the classifier itself.

To reuse fitted data transformers (preprocessors, decomposition etc), you can **cache pipeline on a disk using `memory=...` argument of the pipeline constructor**.

Whenever you call `.fit(...)` with different inputs the pipeline transformers will be cached:

In [ ]:
import glob
import os
import tempfile
from pprint import pprint


# Utility function to recursively get file names in a folder.
def get_fns_rec(path):
    for f in glob.glob(f"{path}/**/*.*", recursive=True):
        yield os.path.relpath(f, path)


tempdir = tempfile.mkdtemp()
print("Caching to directory:", tempdir)

# memory argument takes a path to a directory
# (or a `joblib.Memory` object thar represent a directory)
p = make_pipeline(StandardScaler(), PCA(3), SVC(), memory=tempdir)

print("Files before fit:")
pprint(list(get_fns_rec(tempdir)))
print()

p.fit(features, labels)
print("Files after fit 1:")
pprint(list(get_fns_rec(tempdir)))
print()

Each data transformer in our pipeline has a corresponding:
* a "pickle" (binary `.pkl` file) containing serialized transformer object, and
* a metadata file (human-readable `.json` file) containing 1) description of the transformer constructor and 2) input data used for training - both of these identify a cached transformer.

Cache is extended on fit with different input data and re-used on fit with the same input data:

In [ ]:
print("Files after fit 2:")
# alter input data by dropping first 10 samples
p.fit(features.iloc[:-10, :], labels.iloc[:-10])
pprint(list(get_fns_rec(tempdir)))
print()

# if inputs are the same - cache will be used
print("Files after fit 1 repeat:")
p.fit(features, labels)
pprint(list(get_fns_rec(tempdir)))
print()

<div class="alert alert-block alert-info"><p>
    <i class="fa fa-info-circle"></i>&nbsp;<strong>Beware</strong> that creating a pipeline with <code>memory=...</code> argument clones the transformer objects passed in a constructor. In turn, to inspect fitted transformers you must use <code>steps</code> or <code>named_steps</code> attribute of the pipeline object and not a previous explicit reference to a transformer used in pipeline (in particular, previous references won't be fitted after the pipeline was fitted).
</p></div>

<div class="alert alert-success" role="alert">

## Take-home message: Pipelines

- **scikit-learn API**: All classifiers implement `.fit` and `.predict` methods, all transformers implement `.fit`, `.transform` and `.fit_transform` methods &rarr; allows easy exchange of transformers and classifiers.
- **Building blocks of pipeline**: 0 or more data transformers plus a final classifier.
- **Why and how can pipelines be reused?**: To save time pipelines can be stored for evaluiation using the standard library `pickle` for serialization and deserialization.
- **When is caching of data pipelines useful**: When reusing preprocessing steps already fitted to data.
    
<p>
</div>

## Hyperparameter optimization

Classifiers and pipelines have parameters which must be adapted for improving performance (e.g. `gamma` or `C`). Finding good parameters is also called *hyperparameter optimization* to distinguish from the optimization done during learning of many classification algorithms.

<br/>
<div style="font-size: 120%;">Up to now we adapted such hyperparameters manually, but there are more systematic approaches !</div>

### Grid search

The simplest approach is to **specify valid values for each hyperparameter involved in training and then try out all possible combinations**. This is called *grid search*. In `scikit-learn` grid search is by default combined with cross-validation to asses classifier's performance for each hyperparameter set:

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

beer_data = pd.read_csv("data/beers.csv")

features = beer_data.iloc[:, :-1]
labels = beer_data.iloc[:, -1]

svc = SVC()

# optimize hyperparameters of one single classifier

# the keys in the dictionary match the argument names for SVC:
parameters = {"kernel": ("linear", "rbf", "poly"), "C": [1, 5, 10, 15]}

# run gridsearch, use CV to assess quality and determine best parameter
# set based on classification accuracy score:

# tries all 3 x 4 = 12 combinations of hyperparameter values:
search = GridSearchCV(svc, parameters, cv=4, scoring="accuracy")
search.fit(features, labels)

print(search.best_score_, search.best_params_)

Such an optimization can also be applied to a full pipeline:

In [ ]:
import tempfile

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

# using temporary transfomers caching for a bit better speed
cache_dir = tempfile.mkdtemp()

p = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(),
    LogisticRegression(max_iter=10000),
    memory=cache_dir,
)

The specification of the grid id now a bit more complicated `PROCESSOR__ARGUMENT`: 

- first the name of the processor / classifier in lower case letters,
- then two underscores `__`,
- finally the name of the argument of the processor / classifier.

`StandardScaler` e.g. has parameters `with_mean` and `with_std` which can be `True` or `False`:

In [ ]:
param_grid = {
    "standardscaler__with_mean": [True, False],
    "standardscaler__with_std": [True, False],
    "polynomialfeatures__degree": [1, 2, 3, 4],
    "logisticregression__C": [0.01, 0.1, 1, 10, 100],
}

This grid has `4 x 2 x 2 x 5` thus `80` points. So we muss run crossvalidation for 80 different classifiers.

To speed this up, we can specify `n_jobs = 2` to use `2` extra processor cores to run gridsearch in parallel (you might want to use more cores depending on your computer):

In [ ]:
search = GridSearchCV(
    p,
    param_grid,
    cv=4,
    scoring="accuracy",
    n_jobs=2,
)
search.fit(features, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

If you have more complicated pipelines, you also can assign explicitly names to the steps which can be used in the parameter grid. However, you need to use directly the `Pipeline(...)` constructor to do so:

In [ ]:
from sklearn.pipeline import Pipeline

p_names = Pipeline(
    steps=[
        ("scale", StandardScaler()),
        ("poly", PolynomialFeatures()),
        ("clf", LogisticRegression(max_iter=10000)),
    ],
    # memory=cache_dir,
)

param_grid_short_names = {
    "scale__with_mean": [True, False],
    "scale__with_std": [True, False],
    "poly__degree": [1, 2, 3, 4],
    "clf__C": [0.01, 0.1, 1, 10],
}

import time

last = 1.0
for n_jobs in (2, 3, 4, 5, 6, 7):

    s = time.time()
    for _ in range(5):
        search = GridSearchCV(
            p_names,
            param_grid_short_names,
            cv=4,
            scoring="accuracy",
            n_jobs=n_jobs,
        )

        search.fit(features, labels)
    needed = time.time() - s

    print(f"{n_jobs} {needed:7.1f} s   {needed/last:5.2f}")
    last = needed

# using a built-in notebook line magic to measure the fit time
%time search.fit(features, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

### Random search

These grid searches took some time. A less systematic but a much more efficient approach for a rather well performing classifier is *random search*.

In this case, instead of a whole grid, we **specify probability distributions for the hyperparameters to draw random samples from**:

In [ ]:
from scipy.stats import loguniform, randint

param_dist_short_names = {
    "scale__with_mean": [True, False],  # random value from explicit set of values
    "scale__with_std": [True, False],
    "poly__degree": randint(1, 4),  # random integer from 1 to 4
    "clf__C": loguniform(0.01, 100),  # log random number from .01 to 100
}

We run now 30 iterations.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

search = RandomizedSearchCV(
    p_names,
    param_dist_short_names,
    cv=4,
    scoring="accuracy",
    n_jobs=2,
    n_iter=30,
    random_state=21,  # fix randomization for reproduciblity
)
%time search.fit(features, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

<div class="alert alert-block alert-warning">
<p><i class="fa fa-info-circle"></i>
Hyperparameter search methods also "behave" like a single classifier - they implement <code>.fit()</code> and <code>.predict()</code> methods (*).</p>
</div>

<div>
<p>(*) Prediction is done with the best parameters found. The underlying model or pipeline with the best parameters is available via <code>.best_estimator_</code> property. Importantly, the <strong>refit with the best parameters is done at the end</strong> of the CV-based search, <strong>using a whole training data set</strong>.</p>
    
<p style="font-size: 80%;">The automatic refitting can be disabled by passing <code>refit=False</code> argument when specifying the search method. Then neither <code>.predict()</code>, nor <code>.best_estimator_</code> won't be available.</p>
</div>





In [ ]:
print("Best estimator:")
print(search.best_estimator_)
print()
print()
print("Training set accuracy:", sum(search.predict(features) == labels) / len(labels))

### Halving searches

The `scikit-learn` includes also a drop-in replacements for both cross-validation searches that **start search at each hyperparameters set with only small random subset of samples and iteratively selects best results while increasing the number of samples**: `HalvingGridSearchCV` and `HalvingRandomSearchCV`. This is called a *halving search* technique.

<table>
    <tr><td><img src="images/halving_search.png" width="70%"/></td></tr>
    <tr><td><center><sub>Source: <a href="https://scikit-learn.org/stable/auto_examples/model_selection/plot_successive_halving_iterations.html#sphx-glr-auto-examples-model-selection-plot-successive-halving-iterations-py">https://scikit-learn.org/stable/auto_examples/model_selection/plot_successive_halving_iterations.html#sphx-glr-auto-examples-model-selection-plot-successive-halving-iterations-py</a></sub></center></td></tr>
</table>

The number of samples is just a default *resource* to increase in iterations. Resource can specified alternatively, e.g., as a number of iterations that the final classifier runs while learning.

For large hyperparameter space/grid the results usually are as accurate as for standard searches, but found in much less time:

In [ ]:
# halving searches are still an experimental feature (scikit-learn 1.1.2)
# => needs to be explicitly enabled via following import:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

search = HalvingGridSearchCV(
    p_names,
    param_grid_short_names,
    cv=4,
    scoring="accuracy",
    n_jobs=2,
    factor=2,  # default is to actually third, not halve
    random_state=21,  # fix randomization for reproduciblity
)
%time search.fit(features, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

<div class="alert alert-success" role="alert">

## Take-home message: Hyperparameter optimization

- **Grid search**: Test all combinations of specified values of hyperparameters.
- **Random search**: Specify probability distributions per hyperparameter and draw random samples.
- **Halving searches**: Start with small number of samples and all candidates and over iterations reduce number of candidates (always choose best ones) and increase number of samples until only one candidate is left.

<p>
</div>

## Exercise session

1. Try to find good parameters for the following two pipelines applied to the beer data set. Use grid search for the first one and randomized search for the second one. Cache the data transformers while doing the grid search. Does it pay-off to cache the data transformers during randomized search? Save the search results to the disk.

    `make_pipeline(StandardScaler(), SVC(gamma=..., C=...), memory=...)`
    
    `make_pipeline(StandardScaler(), PolynomialFeatures(degree=...), PCA(n_components=...), LogisticRegression(C=...))`

In [ ]:
# SOLUTION

import os
import pickle
import tempfile

cache_dir = tempfile.mkdtemp()


beer_data = pd.read_csv("data/beers.csv")

features = beer_data.iloc[:, :-1]
labels = beer_data.iloc[:, -1]


p = make_pipeline(StandardScaler(), SVC(), memory=cache_dir)

param_grid = {
    "standardscaler__with_mean": [True, False],
    "standardscaler__with_std": [True, False],
    "svc__C": [1, 10, 15, 20, 25],
    "svc__gamma": [0.01, 0.05, 0.1, 0.5, 1],
}

search = GridSearchCV(p, param_grid, cv=5, scoring="accuracy", n_jobs=5)
search.fit(features, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

search_path = os.path.join(cache_dir, "search_1.pkl")
with open(search_path, "wb") as search_file:
    pickle.dump(search, search_file)


from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVC

p = make_pipeline(
    StandardScaler(), PolynomialFeatures(), PCA(), LogisticRegression(max_iter=100_000)
)
param_grid = {
    "polynomialfeatures__degree": randint(2, 4),
    "pca__n_components": randint(2, 16),
    "logisticregression__C": loguniform(0.1, 100),
}
# Note: not using cache w/ randomized search for this pipeline.
#       Only the fast StandardScaler would benefit from caching;
#       overhead for caching all transformers w/ random values is bigger.

search = RandomizedSearchCV(
    p,
    param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=5,
    random_state=42,  # fix randomization for reproduciblity
)
search.fit(features, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

search_path = os.path.join(cache_dir, "search_2.pkl")
with open(search_path, "wb") as search_file:
    pickle.dump(search, search_file)

### Optional Exercises

Using random search optimize parameters for pipelines for:

- the spiral data set (`SVC`)
- the digits data set (`PCA` + `SVC`)


In [ ]:
# SOLUTION

import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import loguniform
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

data = pd.read_csv("data/spiral.csv")

features = data.iloc[:, :-1]
labels = data.iloc[:, -1]

# note: data is scaled already
plt.figure(figsize=(5, 5))
plt.scatter(features.iloc[:, 0], features.iloc[:, 1], color=["rb"[l] for l in labels])
labels = data.iloc[:, -1]

clf = SVC()

param_grid = {
    "C": loguniform(0.01, 100),
    "gamma": loguniform(0.01, 100),
}

search = RandomizedSearchCV(
    clf,
    param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=5,
    random_state=21,  # fix randomization for reproduciblity
)
search.fit(features, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
# SOLUTION

import random

from scipy.stats import loguniform, randint, uniform
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

# classifying digits

data_set = load_digits()
features = data_set.data
labels = data_set.target


param_dist = {
    "pca__n_components": randint(15, 20),
    "svc__C": uniform(4, 6),
    "svc__gamma": loguniform(1e-5, 1e-3),
}

p = make_pipeline(PCA(), SVC())

search = RandomizedSearchCV(
    p,
    param_dist,
    cv=5,
    scoring="accuracy",
    n_jobs=4,
    n_iter=40,
    random_state=21,  # fix randomization for reproduciblity
)
search.fit(features, labels)

print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Copyright (C) 2019-2023 ETH Zurich, SIS ID